In [72]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ast import literal_eval
from dataclasses import dataclass
from sympy.combinatorics import Permutation
from typing import Dict, List
import json
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname('/home/xuanming/kaggle/santa/DeepCubeA-master/data/puzzle15/test'))))

In [73]:
puzzle_info = pd.read_csv("./santa-2023/puzzle_info.csv")
puzzles = pd.read_csv("./santa-2023/puzzles.csv")
submission = pd.read_csv("./santa-2023/sample_submission.csv")

In [74]:
puzzles.query("puzzle_type == 'cube_3/3/3'")

,id,puzzle_type,solution_state,initial_state,num_wildcards
30,30,cube_3/3/3,A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;...,A;A;E;C;F;F;C;C;C;D;B;D;D;B;A;B;F;F;A;E;B;D;E;...,0
31,31,cube_3/3/3,A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;...,C;B;E;B;A;B;A;F;F;E;D;D;F;E;A;F;C;B;C;F;B;B;B;...,6
32,32,cube_3/3/3,A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;...,F;B;B;E;E;D;B;E;E;F;A;D;C;D;A;B;E;A;F;C;A;C;F;...,0
33,33,cube_3/3/3,A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;...,C;A;F;D;F;A;B;F;D;F;B;A;C;B;B;F;A;A;C;D;C;E;E;...,6
34,34,cube_3/3/3,A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;C;C;C;C;C;...,B;C;F;F;D;D;C;A;B;A;B;A;E;A;F;E;B;C;E;C;D;D;C;...,0
...,...,...,...,...,...
145,145,cube_3/3/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N15;N19;N53;N21;N49;N3;N9;N34;N51;N38;N52;N42;...,2
146,146,cube_3/3/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N15;N43;N53;N25;N49;N10;N27;N14;N0;N20;N21;N29...,0
147,147,cube_3/3/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N8;N14;N0;N41;N22;N25;N15;N30;N38;N45;N23;N9;N...,0
148,148,cube_3/3/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N26;N28;N9;N5;N40;N48;N18;N37;N0;N11;N3;N29;N2...,0


In [76]:
submission

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [87]:
sum(submission['moves'].apply(lambda x:len(x.split('.'))).tolist())

1220590

In [78]:
puzzle_info

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [79]:
puzzles

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_33/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0


In [81]:
types = puzzle_info['puzzle_type'].unique()
# for t in types:
#     tmp = set(puzzles.query('puzzle_type == @t').apply(lambda x: ";".join(sorted(set(x.solution_state.split(';')))), axis = 1))
#     print(t, '\n',len(tmp),'\n', tmp)
#     print('===' * 30)

In [90]:
allowed_moves = {}
for t in types:
    allowed_moves[t] = eval(puzzle_info.query("puzzle_type == @t")['allowed_moves'].values[0])

In [91]:
inverse_allowed_moves = {}
for t in allowed_moves:
    inverse_allowed_moves[t] = {}
    for k in allowed_moves[t]:
        tmp = [[s, idx] for idx, s in enumerate(allowed_moves[t][k])]
        tmp.sort()
        tmp = [b for a, b in tmp]
        inverse_allowed_moves[t][k] = tmp 

In [94]:
for t in inverse_allowed_moves:
    for k in inverse_allowed_moves[t]:
        allowed_moves[t]['-' + k] = inverse_allowed_moves[t][k] 

dict_keys(['f0', 'f1', 'r0', 'r1', 'd0', 'd1', '-f0', '-f1', '-r0', '-r1', '-d0', '-d1'])

In [101]:
import pickle
with open('allowed_moves.pickle', 'wb') as handle:
    pickle.dump(allowed_moves, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('inverse_allowed_moves.pickle', 'wb') as handle:
    pickle.dump(inverse_allowed_moves, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
load_path = "/home/xuanming/kaggle/santa/allowed_moves.pickle"
with open(load_path, 'rb') as f:
    new_dict = pickle.load(f)

In [78]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname('/home/xuanming/kaggle/santa/DeepCubeA-master/data/puzzle15/test'))))

In [32]:
load_path = "/home/xuanming/kaggle/santa/DeepCubeA-master/data/cube3/test/data_0.pkl"
with open(load_path, 'rb') as f:
    test_state = pickle.load(f)

In [33]:
test_state['states']

In [4]:
import pickle

In [5]:
from environments.santa import SantaState, Santa
load_path = "/home/xuanming/kaggle/santa/allowed_moves.pickle"
with open(load_path, 'rb') as f:
    allowed_moves = pickle.load(f)
load_path = "/home/xuanming/kaggle/santa/inverse_allowed_moves.pickle"
with open(load_path, 'rb') as f:
    inverse_allowed_moves = pickle.load(f)

goal= np.array(['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D',
'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F'])
env = Santa(goal, allowed_moves['cube_2/2/2'], inverse_allowed_moves['cube_2/2/2'])

In [42]:
sample = SantaState(np.array(puzzles.loc[0]['initial_state'].split(';')), 0)

In [43]:
np.array(puzzles.loc[0]['initial_state'].split(';'))

array(['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D',
       'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C'], dtype='<U1')

In [11]:
inital_state = sample

In [12]:
env.next_state([sample], 'f0')[0][0].state

array(['D', 'E', 'E', 'F', 'A', 'E', 'B', 'B', 'D', 'A', 'A', 'A', 'D',
       'C', 'D', 'F', 'F', 'B', 'E', 'F', 'C', 'C', 'B', 'C'], dtype='<U1')

In [60]:
next_state = env.next_state([sample], 'f0')
next_state = env.next_state(next_state[0], 'f0')
next_state = env.next_state(next_state[0], 'f0')

In [59]:
next_state[0][0].state

array(['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D',
       'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C'], dtype='<U1')

In [18]:
env.prev_state(next_state[0], 'f0')[0].state

array(['D', 'E', 'D', 'A', 'E', 'B', 'A', 'B', 'C', 'A', 'C', 'A', 'D',
       'C', 'D', 'F', 'F', 'F', 'E', 'E', 'B', 'F', 'B', 'C'], dtype='<U1')

In [24]:
puzzles

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_33/3,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_33/3,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0


In [31]:
santa_test = {'states':[]}
for idx, row in puzzles.query("puzzle_type == 'cube_2/2/2'").iterrows():
    if 'N0' in row['initial_state'].split(';'):
        sample = SantaState(np.array([int(x[1:]) for x in row['initial_state'].split(';')]), row['num_wildcards'], np.array([int(x[1:]) for x in row['solution_state'].split(';')]))
        santa_test['states'].append(sample)

In [32]:
with open('./DeepCubeA-master/data/santa/test/data_0.pkl', 'wb') as handle:
    pickle.dump(santa_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
santa_test['states'][0].goal_state

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [ ]:
puzzles

In [34]:
load_path = "/home/xuanming/kaggle/santa/DeepCubeA-master/results/santa/results.pkl"
with open(load_path, 'rb') as f:
    res = pickle.load(f)

In [41]:
submission

,id,moves
0,0,r1.-f1
1,1,f1.d0.-r0.-f1.-d0.-f1.d0.-r0.f0.-f1.-r0.f1.-d1...
2,2,f1.d0.-d1.r0.-d1.-f0.f1.-r0.-f0.-r1.-f0.r0.-d0...
3,3,-f0.-r0.-f0.-d0.-f0.f1.r0.-d1.-r0.-r1.-r0.-f1....
4,4,d1.-f1.d1.r1.-f0.d1.-d0.-r1.d1.d1.-f1.d1.-d0.-...
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [66]:
res['solutions']

[[2, 0, 0, 5, 0, 2, 1, 2, 4, 1, 2, 5],
 [1, 2, 2, 4, 0, 3, 0, 5, 2, 5, 2, 4],
 [3, 3, 1, 3, 0, 5, 3, 4, 3, 3, 5],
 [4, 0, 3, 0, 5, 5, 0, 3, 5, 1, 5],
 [2, 5, 0, 0, 2, 4, 2, 1, 2, 5, 5]]

In [64]:
for r in res['solutions']:
    print(len(r))

12
12
11
11
11
